### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2

#### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Алламе Ормиз
##### Группа:  НФИбд-01-21

## Москва 2024
***

### Вариант № 6

Для закрепленного за Вами варианта лабораторной работы:

1. При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db. 

In [1]:
import sqlite3

db_path = 'instacart.db'

conn = sqlite3.connect(db_path)

print("База данных успешно открыта")

База данных успешно открыта


2. Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме, количество транзакций (покупок) и определите количество товаров (столбец product_id) в транзакциях датафрейма. 

In [2]:
import pandas as pd

departments_df = pd.read_sql_query("SELECT * FROM departments", conn)
products_df = pd.read_sql_query("SELECT * FROM products", conn)

a = 0
b = 8

query = '''
    SELECT
      opt.order_id, 
      opt.product_id, 
      add_to_cart_order, 
      reordered, 
      product_name
    FROM order_products__train opt
    JOIN orders o ON opt.order_id = o.order_id
    JOIN products p ON opt.product_id = p.product_id
    WHERE o.order_dow = ? AND p.department_id = ?
'''

df = pd.read_sql_query(query, conn, params=(a, b))
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,318441,40102,10,1,Nutrisca Chicken And Chickpea Recipe Dog Food
1,2133144,54,16,1,24/7 Performance Cat Litter
2,812040,7518,14,1,Chicken Flavored Bone 4 Inch Dog Chew
3,1307498,26893,11,0,Odor Shield Scented Scoopable Cat Litter
4,1307498,32005,12,0,Grilled Seafood Feast in Gravy Cat Food


In [3]:
num_rows = len(df)
num_transactions = df['order_id'].nunique()
num_products = df['product_id'].nunique() 

print(f"Количество строк: {num_rows}")
print(f"Количество уникальных транзакций: {num_transactions}")
print(f"Количество уникальных товаров: {num_products}")

Количество строк: 888
Количество уникальных транзакций: 551
Количество уникальных товаров: 374


3. Выполните к датафрейму запрос, указанный в индивидуальном задании.

In [4]:
purchased_twice = df.groupby('product_name').size()[df.groupby('product_name').size() == 2].index

print("Список товаров, которые были приобретены ровно два раза:")
print(purchased_twice)

Список товаров, которые были приобретены ровно два раза:
Index(['174 ; Dentastix&#174 ; Large', '6\" Free Ranger Bully Stix',
       'Adult Dog Food', 'Beef Feast in Roasted Beef Flavor Gravy Cat Food',
       'Beggin' Strips Bacon Flavor Dog Snacks',
       'Beyond Grain Free Wild Salmon Recipe Wet Cat Food',
       'Beyond Natural Cat Food Salmon & Whole Brown Rice Recipe',
       'Buddy Biscuits Original Oven Baked Roasted Chicken Dog Treats',
       'Chewy Chicken Flavor Dog Treats',
       'Chicken & Brown Rice Premium Dog Food',
       'Chicken & Vegetable Snack Sticks for Dogs',
       'Chicken Flavored Bone 4 Inch Dog Chew', 'Chicken Jerky Dog Treats',
       'Chicken Liver Paté Canned Cat Food',
       'Chopped Blends With Beef, Carrots, Peas & Barley Dog Food',
       'Chunky Turkey Feast Cat Food',
       'Classic Tender Beef & Chicken Feast Cat Food',
       'Clump & Seal Multi-Cat Cat Litter',
       'Delights With Cheddar Grilled Tuna & Cheddar Cheese Feast In Gravy',
   

4. Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных транзакцию с наибольшим количеством товаров и выведите ее на экран.  

In [5]:
import pandas as pd

transactions = df.groupby('order_id')['product_name'].apply(list).reset_index()

transaction_counts = df.groupby('order_id')['product_name'].count()

max_count = transaction_counts.max()

max_transactions = transactions[transactions['order_id'].isin(transaction_counts[transaction_counts == max_count].index)]

print("Транзакции с наибольшим количеством товаров:")
print(max_transactions)

Транзакции с наибольшим количеством товаров:
    order_id                                       product_name
466   506992  [Classic Tender Beef Feast Cat Food, Flaked Tu...


5. Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите и выведите на экран три наиболее популярных товара и определите количество покупок (транзакций) этих товаров. 

In [6]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions['product_name']).transform(transactions['product_name'])
df_binary = pd.DataFrame(te_ary, columns=te.columns_)

item_counts = df_binary.sum().sort_values(ascending=False)

top_3_items = item_counts.head(3)

print("Три наиболее популярных товара и количество их покупок:")
print(top_3_items)

Три наиболее популярных товара и количество их покупок:
Instant Action Cat Litter              23
24/7 Performance Cat Litter            18
Grain Free Chicken Formula Cat Food    17
dtype: int64


6. При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте и выведите на экран популярный набор предметов с (абсолютной) минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии такого набора уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярного набора предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора). 

In [7]:
from mlxtend.frequent_patterns import fpgrowth

itemsets = fpgrowth(df_binary, min_support=3/df_binary.shape[0],use_colnames=True)
itemsets

,support,itemsets
0,0.016334,(Snack Sticks Chicken & Rice Recipe Dog Treats)
1,0.016334,(Classic Tender Beef Feast Cat Food)
2,0.007260,(Fancy Feast Wet Classic Turkey & Giblets Feas...
3,0.007260,(Scented Multi-Cat Scoopable Cat Litter)
4,0.009074,"(Dog Snacks, Medium)"
...,...,...
116,0.005445,"(Grain Free Turkey Formula Cat Food, Natural B..."
117,0.005445,(Classic Tender Liver & Chicken Feast Cat Food...
118,0.005445,(Classic Tender Liver & Chicken Feast Cat Food...
119,0.005445,(Classic Tender Liver & Chicken Feast Cat Food...


In [8]:
max_length = itemsets['itemsets'].apply(lambda x: len(x)).max()
max_length_itemsets = itemsets[itemsets['itemsets'].apply(lambda x: len(x)) == max_length]

print(max_length_itemsets)

      support                                           itemsets
115  0.005445  (Grain Free Chicken Formula Cat Food, Grain Fr...


7. Используя пакет mlxtend или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов с уровнем достоверности (confidence) не ниже $0.6$. 

In [9]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(itemsets, metric="confidence", min_threshold=0.6)

print("Ассоциативные правила с уровнем достоверности не ниже 0.6:")
rules

Ассоциативные правила с уровнем достоверности не ниже 0.6:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Fancy Feast Wet Classic Turkey & Giblets Feas...,(Classic Tender Beef Feast Cat Food),0.007260,0.016334,0.005445,0.750000,45.916667,0.005326,3.934664,0.985375
1,(Grilled Turkey Feast in Gravy Cat Food),(Grilled Chicken Feast in Gravy Cat Food),0.009074,0.010889,0.007260,0.800000,73.466667,0.007161,4.945554,0.995421
2,(Grilled Chicken Feast in Gravy Cat Food),(Grilled Turkey Feast in Gravy Cat Food),0.010889,0.009074,0.007260,0.666667,73.466667,0.007161,2.972777,0.997248
3,(Grilled Turkey Feast in Gravy Cat Food),(Fancy Feast Wet Classic Chicken Feast Cat Food),0.009074,0.018149,0.005445,0.600000,33.060000,0.005280,2.454628,0.978632
4,(Grain Free Turkey Formula Cat Food),(Grain Free Turkey & Salmon Formula Cat Food),0.014519,0.021779,0.009074,0.625000,28.697917,0.008758,2.608590,0.979374
5,"(Grain Free Chicken Formula Cat Food, Grain Fr...",(Grain Free Turkey & Salmon Formula Cat Food),0.007260,0.021779,0.005445,0.750000,34.437500,0.005287,3.912886,0.978062
6,"(Grain Free Chicken Formula Cat Food, Grain Fr...",(Grain Free Turkey Formula Cat Food),0.009074,0.014519,0.005445,0.600000,41.325000,0.005313,2.463702,0.984737
7,"(Grain Free Turkey Formula Cat Food, Grain Fre...",(Grain Free Chicken Formula Cat Food),0.009074,0.030853,0.005445,0.600000,19.447059,0.005165,2.422868,0.957265
8,(Natural Beef & Chicken Formula Food for Cats),(Grain Free Turkey Formula Cat Food),0.007260,0.014519,0.005445,0.750000,51.656250,0.005339,3.941924,0.987812
9,(Classic Ocean Whitefish & Tuna Feast Cat Food),(Classic Tender Liver & Chicken Feast Cat Food),0.007260,0.012704,0.005445,0.750000,59.035714,0.005352,3.949183,0.990250


8. Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и выведите на экран ассоциативное правило, имеющее наилучшее значение этого показателя оценки. 

In [11]:
best_rule = rules.loc[rules['lift'].idxmax()]
best_rule = rules.loc[rules['lift'].idxmax()]

print("Ассоциативное правило с наилучшим значением:")
best_rule

Ассоциативное правило с наилучшим значением:


antecedents           (Chunky Beef with Vegetables & Brown Rice Dog ...
consequents           (Select Tender Chicken with Vegetables & Brown...
antecedent support                                              0.00726
consequent support                                              0.00726
support                                                        0.005445
confidence                                                         0.75
lift                                                           103.3125
leverage                                                       0.005392
conviction                                                     3.970962
zhangs_metric                                                  0.997562
Name: 10, dtype: object